Scheduler Development Notebook

In [8]:
# Schedule.py Pulls forecast data from the BOM each day and saves it in an SQL db.

import pandas as pd # Structure and Dataframes
import datetime as dt # Time Functions
import requests # API fetching

# SQL & Credientials Mgnt
import sqlalchemy
import os
import dotenv # Protect db creds
dotenv.load_dotenv()

True

In [9]:
# SQL Connection
DATABASE_URL = os.environ.get('DATABASE_URL')
engine = sqlalchemy.create_engine(DATABASE_URL)
db = pd.read_sql('bom-weather', engine)

In [10]:
db.head()

,date,extended_text,fire_danger,forecast,icon_descriptor,issue,location,rain_max,rain_min,short_text,sunrise,sunset,temp_max,temp_min,uv_cat,uv_index
0,2020-10-27T13:00:00Z,Partly cloudy. Patchy morning fog. Medium (40%...,Low-Moderate,0,shower,2020-10-28,Melbourne,1.0,0,Possible afternoon shower.,2020-10-27T19:19:10Z,2020-10-28T08:51:13Z,20,10,high,7.0
1,2020-10-28T13:00:00Z,Partly cloudy. The chance of fog in the early ...,Low-Moderate,1,shower,2020-10-28,Melbourne,2.0,0,Possible shower developing.,2020-10-28T19:17:59Z,2020-10-29T08:52:17Z,21,10,veryhigh,8.0
2,2020-10-29T13:00:00Z,Partly cloudy. The chance of morning fog in th...,Low-Moderate,2,shower,2020-10-28,Melbourne,3.0,0,Shower or two.,2020-10-29T19:16:49Z,2020-10-30T08:53:21Z,20,12,veryhigh,8.0
3,2020-10-30T13:00:00Z,Cloudy. Medium (60%) chance of showers. Winds ...,Low-Moderate,3,shower,2020-10-28,Melbourne,3.0,0,Shower or two.,2020-10-30T19:15:41Z,2020-10-31T08:54:25Z,18,12,veryhigh,8.0
4,2020-10-31T13:00:00Z,Cloudy. Slight (30%) chance of a shower. Winds...,None,4,cloudy,2020-10-28,Melbourne,0.2,0,Cloudy.,2020-10-31T19:14:34Z,2020-11-01T08:55:29Z,19,12,None,NaN


In [11]:
# Define Reference Times
today = dt.date.today()
todaystr = today.strftime("%Y-%m-%d")

In [13]:
print(today,todaystr)

2020-10-29 2020-10-29


In [12]:
# Forecast Locations
## More URL's can be found via https://weather.bom.gov.au/search & talking the location reference from the URL
## Eg: https://weather.bom.gov.au/location/r1r5rjm-clonbinane << 'r1r5rjm'
locations = {
'Melbourne' : 'https://api.weather.bom.gov.au/v1/locations/r1r143/forecasts/daily',
} 

## Finished Loop 

In [14]:
# Fetch Location from location dicts.
for name,url in locations.items():
    response = requests.get(url)
    weather_dict = response.json() # API Forecast as json
    af = pd.DataFrame(weather_dict['data']) # Dict has 'data' and 'meta'
    # EDA
    # Set forecast Dates
    af['issue'] = today
    af['forecast'] = [str(i) for i in range(len(af))]
    # Set Location
    af['location'] = name
    # Split rain
    af['rain_min'] = [row['amount']['min'] for row in af['rain']]
    af['rain_max'] = [row['amount']['max'] for row in af['rain']]
    af['rain_max'].fillna(0,inplace=True) # Rain Max is na if no rain forecast.
    # Split UV
    af['uv_cat'] = [row['category'] for row in af['uv']]
    af['uv_index'] = [row['max_index'] for row in af['uv']]
    # Split astronomical (Note: times are UTC)
    af['sunrise'] = [row['sunrise_time'] for row in af['astronomical']]
    af['sunset'] = [row['sunset_time'] for row in af['astronomical']]
    # Clean Up
    af.drop(['rain','uv','astronomical','now'],axis=1,inplace=True)
    af = af.reindex(sorted(af.columns), axis=1)
    # Add new data to forecast and push back into DB
#     db.append(af)
#     db.drop_duplicates().reset_index(drop=True) # In the case of pulling x2 in one day.
#     db.to_sql('bom-weather', engine, if_exists = 'append', index=False)
print('Done')

Done


In [15]:
af







,date,extended_text,fire_danger,forecast,icon_descriptor,issue,location,rain_max,rain_min,short_text,sunrise,sunset,temp_max,temp_min,uv_cat,uv_index
0,2020-10-28T13:00:00Z,"Cloudy. Medium (50%) chance of showers, becomi...",None,0,shower,2020-10-29,Melbourne,2.0,0,Possible shower.,2020-10-28T19:17:59Z,2020-10-29T08:52:17Z,22,NaN,None,NaN
1,2020-10-29T13:00:00Z,Cloudy. Areas of morning fog. High (70%) chanc...,Low-Moderate,1,shower,2020-10-29,Melbourne,2.0,0,Shower or two.,2020-10-29T19:16:49Z,2020-10-30T08:53:21Z,20,13.0,veryhigh,8.0
2,2020-10-30T13:00:00Z,Partly cloudy. The chance of early fog about n...,Low-Moderate,2,shower,2020-10-29,Melbourne,0.4,0,Shower or two.,2020-10-30T19:15:41Z,2020-10-31T08:54:25Z,18,13.0,veryhigh,9.0
3,2020-10-31T13:00:00Z,Partly cloudy. The chance of morning fog in th...,Low-Moderate,3,cloudy,2020-10-29,Melbourne,0.0,0,Partly cloudy.,2020-10-31T19:14:34Z,2020-11-01T08:55:29Z,19,11.0,veryhigh,8.0
4,2020-11-01T13:00:00Z,Patchy morning fog. Mostly sunny day. Light wi...,Low-Moderate,4,mostly_sunny,2020-10-29,Melbourne,0.0,0,Mostly sunny.,2020-11-01T19:13:28Z,2020-11-02T08:56:34Z,26,9.0,veryhigh,9.0
5,2020-11-02T13:00:00Z,Sunny. Winds northerly 15 to 25 km/h.,None,5,mostly_sunny,2020-10-29,Melbourne,0.0,0,Sunny.,2020-11-02T19:12:24Z,2020-11-03T08:57:39Z,28,16.0,None,NaN
6,2020-11-03T13:00:00Z,Partly cloudy. High (70%) chance of showers. W...,None,6,shower,2020-10-29,Melbourne,3.0,0,Shower or two.,2020-11-03T19:11:21Z,2020-11-04T08:58:45Z,21,18.0,None,NaN
7,2020-11-04T13:00:00Z,Partly cloudy. Medium (40%) chance of showers....,None,7,shower,2020-10-29,Melbourne,1.0,0,Possible shower.,2020-11-04T19:10:20Z,2020-11-05T08:59:50Z,16,10.0,None,NaN


In [42]:
db2 = db.copy()

In [43]:
db2 = db2.append(af)

In [44]:
db2.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)  #.reset_index(drop=True)

In [45]:
len(db2)

15

In [46]:
db2

,date,extended_text,fire_danger,forecast,icon_descriptor,issue,location,rain_max,rain_min,short_text,sunrise,sunset,temp_max,temp_min,uv_cat,uv_index
0,2020-10-27T13:00:00Z,Partly cloudy. Patchy morning fog. Medium (40%...,Low-Moderate,0,shower,2020-10-28,Melbourne,1.0,0,Possible afternoon shower.,2020-10-27T19:19:10Z,2020-10-28T08:51:13Z,20,10.0,high,7.0
1,2020-10-28T13:00:00Z,Partly cloudy. The chance of fog in the early ...,Low-Moderate,1,shower,2020-10-28,Melbourne,2.0,0,Possible shower developing.,2020-10-28T19:17:59Z,2020-10-29T08:52:17Z,21,10.0,veryhigh,8.0
2,2020-10-29T13:00:00Z,Partly cloudy. The chance of morning fog in th...,Low-Moderate,2,shower,2020-10-28,Melbourne,3.0,0,Shower or two.,2020-10-29T19:16:49Z,2020-10-30T08:53:21Z,20,12.0,veryhigh,8.0
3,2020-10-30T13:00:00Z,Cloudy. Medium (60%) chance of showers. Winds ...,Low-Moderate,3,shower,2020-10-28,Melbourne,3.0,0,Shower or two.,2020-10-30T19:15:41Z,2020-10-31T08:54:25Z,18,12.0,veryhigh,8.0
4,2020-10-31T13:00:00Z,Cloudy. Slight (30%) chance of a shower. Winds...,None,4,cloudy,2020-10-28,Melbourne,0.2,0,Cloudy.,2020-10-31T19:14:34Z,2020-11-01T08:55:29Z,19,12.0,None,NaN
5,2020-11-01T13:00:00Z,Partly cloudy. The chance of morning fog. Ligh...,None,5,mostly_sunny,2020-10-28,Melbourne,0.0,0,Mostly sunny.,2020-11-01T19:13:28Z,2020-11-02T08:56:34Z,24,11.0,None,NaN
6,2020-11-02T13:00:00Z,Mostly sunny. Winds northerly 15 to 25 km/h.,None,6,mostly_sunny,2020-10-28,Melbourne,0.0,0,Mostly sunny.,2020-11-02T19:12:24Z,2020-11-03T08:57:39Z,27,15.0,None,NaN
7,2020-10-28T13:00:00Z,"Cloudy. Medium (50%) chance of showers, becomi...",None,0,shower,2020-10-29,Melbourne,2.0,0,Possible shower.,2020-10-28T19:17:59Z,2020-10-29T08:52:17Z,22,NaN,None,NaN
8,2020-10-29T13:00:00Z,Cloudy. Areas of morning fog. High (70%) chanc...,Low-Moderate,1,shower,2020-10-29,Melbourne,2.0,0,Shower or two.,2020-10-29T19:16:49Z,2020-10-30T08:53:21Z,20,13.0,veryhigh,8.0
9,2020-10-30T13:00:00Z,Partly cloudy. The chance of early fog about n...,Low-Moderate,2,shower,2020-10-29,Melbourne,0.4,0,Shower or two.,2020-10-30T19:15:41Z,2020-10-31T08:54:25Z,18,13.0,veryhigh,9.0
